In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [31]:
# parser v1
# parse records to list of dicts
# values mostly are strings
# easy to iterate through records
records = []
with open('test.txt') as f:
    for line in f.readlines():
        components = line.strip().split(' ')
        record = {}
        record['timestamp'] = components[0]
        record['displayed'] = components[1]
        record['clicked'] = int(components[2])
        # not using user feature
        record['user_features'] = []
        for i in range(4, len(components)):
            if components[i].startswith('|'):
                n = i
                break
            record['user_features'].append(int(components[i]))
        record['pool'] = components[n:]    
        records.append(record)

In [32]:
records[0]

{'clicked': 0,
 'displayed': 'id-560620',
 'pool': ['|id-552077',
  '|id-555224',
  '|id-555528',
  '|id-559744',
  '|id-559855',
  '|id-560290',
  '|id-560518',
  '|id-560620',
  '|id-563115',
  '|id-563582',
  '|id-563643',
  '|id-563787',
  '|id-563846',
  '|id-563938',
  '|id-564335',
  '|id-564418',
  '|id-564604',
  '|id-565364',
  '|id-565479',
  '|id-565515',
  '|id-565533',
  '|id-565561',
  '|id-565589',
  '|id-565648',
  '|id-565747',
  '|id-565822'],
 'timestamp': '1317513291',
 'user_features': [1,
  9,
  11,
  13,
  23,
  16,
  18,
  17,
  19,
  15,
  43,
  14,
  39,
  30,
  66,
  50,
  27,
  104,
  20]}

In [29]:
d={}

In [33]:
len(records)

148

In [88]:
max(X.values(), key= lambda x:x[0]/x[1])

array([1, 1])

In [62]:
# offline simulation for AG_L
def AG_L(n, m, life_threshold=.3, records):
    # records should be long enough
    '''
    n: number of turns to play
    m: number of turns to initialize
    '''
    global lives # dict of articles to their lifespans
    reward = 0
    i = 0
    X = {} # expectations
    t = 1
    
    # initialization
    while t<=m:
        displayed = records[i]['displayed']
        if displayed not in X.keys():
            X[displayed] = np.array([records[i]['clicked'], 1])
            t += 1            
        '''clicked = records[i]['clicked']
        X[displayed] = X.get(displayed, np.array([0, 0])) + np.array([clicked, 1])'''
        i += 1
    
    while t<=n:
        best = max(X.items(), key= lambda x:x[1][0]/x[1][1])
        X_max = best[1][0]/best[1][1]
        
        if np.random.binomial(1, X_max): # exploit
            while i < len(records): # look for target arm
                if records[i]['displayed'] == get[0]:
                    rewards += records[i]['clicked']
                    t += 1
                    i += 1
                    break
                i += 1
                
        else: # explore
            while i < len(records):               
                born = filter(lambda x: x[1][0]>records[i]['timestamp'], lives.items()) # fileter out unborn articles from lives
                life_rank = sorted(born, key= lambda x: x[1][1], reverse= True) # sort born articles by their expiring date
                young = life_rank[ : int(len(life_rank)*life_threshold)] # articles among top $life_threshold$% on the life ranking
                if records[i]['displayed'] in list(zip(*young)):
                    rewards += records[i]['clicked']
                    t += 1
                    i += 1
                    break
                i += 1
            #np.random.choice(young_arms, 1)
            
    return reward

In [42]:
len(records[0]['pool'])

26

In [2]:
# parser v2
# parse records to dict of lists
# values are mostly strings
# easy to feed in pandas
records = {'timestamp':[],
          'displayed':[],
          'clicked':[],
          'user_features':[],
          'pool':[]}
with open('test.txt') as f:
    for line in f.readlines():
        components = line.strip().split(' ')
        records['timestamp'].append(components[0])
        records['displayed'].append(components[1])
        records['clicked'].append(components[2])
        user_features = []
        for i in range(4, len(components)):
            if components[i].startswith('|'):
                n = i
                break
            user_features.append(int(components[i]))
        records['user_features'].append(user_features)
        records['pool'].append(components[n:])

In [3]:
df = pd.DataFrame(records)
df.head()

,timestamp,displayed,clicked,user_features,pool
0,1317513291,id-560620,0,"[1, 9, 11, 13, 23, 16, 18, 17, 19, 15, 43, 14,...","[|id-552077, |id-555224, |id-555528, |id-55974..."
1,1317513291,id-565648,0,"[1, 9, 11, 13, 16, 15, 14]","[|id-552077, |id-555224, |id-555528, |id-55974..."
2,1317513291,id-563115,0,"[1, 8, 11, 36, 13, 22, 23, 16, 18, 54, 24, 26,...","[|id-552077, |id-555224, |id-555528, |id-55974..."
3,1317513292,id-552077,0,"[1, 7, 11, 37, 13, 23, 16, 18, 17, 35, 15, 14,...","[|id-552077, |id-555224, |id-555528, |id-55974..."
4,1317513292,id-564335,0,[1],"[|id-552077, |id-555224, |id-555528, |id-55974..."


In [7]:
df[df['clicked']=='1']

,timestamp,displayed,clicked,user_features,pool
7,1317513293,id-563938,1,"[1, 13, 23, 33, 18, 24, 17, 19, 29, 14]","[|id-552077, |id-555224, |id-555528, |id-55974..."
13,1317513293,id-564335,1,"[1, 7, 11, 36, 65, 13, 22, 23, 33, 32, 16, 18,...","[|id-552077, |id-555224, |id-555528, |id-55974..."
39,1317513295,id-564335,1,"[1, 8, 12, 36, 65, 13, 22, 23, 16, 18, 24, 26,...","[|id-552077, |id-555224, |id-555528, |id-55974..."
144,1317513299,id-565747,1,"[1, 7, 12, 13, 22, 23, 33, 32, 58, 16, 18, 24,...","[|id-552077, |id-555224, |id-555528, |id-55974..."


In [8]:
df.shape

(148, 5)

In [89]:
# parser that watches article lifespans
lives = {}
with open('Webscope_R6B/ydata-fp-td-clicks-v2_0.20111002') as f:
    for line in f.readlines():
        components = line.strip().split(' ')
        for component in components:
            if component.startswith('|id-'):
                timestamp = int(components[0])
                if component in lives.keys():
                    lives[component][1] = timestamp
                else:
                    lives[component] = [timestamp, timestamp]
                '''if component in lives:
                    lives[component].add(int(components[0]))
                else:
                    lives[component] = set()'''

In [22]:
lives.keys()

dict_keys(['|id-552077', '|id-555224', '|id-555528', '|id-559744', '|id-559855', '|id-560290', '|id-560518', '|id-560620', '|id-563115', '|id-563582', '|id-563643', '|id-563787', '|id-563846', '|id-563938', '|id-564335', '|id-564418', '|id-564604', '|id-565364', '|id-565479', '|id-565515', '|id-565533', '|id-565561', '|id-565589', '|id-565648', '|id-565747', '|id-565822', '|id-565930', '|id-566013', '|id-566022', '|id-566092', '|id-560805', '|id-564371', '|id-562265', '|id-565980', '|id-566431', '|id-566439', '|id-559833', '|id-566541', '|id-562374', '|id-566587', '|id-566478', '|id-566573', '|id-566602', '|id-562637', '|id-566631', '|id-566689', '|id-566726', '|id-566825', '|id-566838'])

In [90]:
for v in lives.values():
    print(v)

[1317513291, 1317515399]
[1317513291, 1317588299]
[1317513291, 1317587099]
[1317513291, 1317515699]
[1317513291, 1317587099]
[1317513291, 1317587099]
[1317513291, 1317578399]
[1317513291, 1317521099]
[1317513291, 1317581999]
[1317513291, 1317583799]
[1317513291, 1317599694]
[1317513291, 1317515699]
[1317513291, 1317599694]
[1317513291, 1317519299]
[1317513291, 1317571199]
[1317513291, 1317599694]
[1317513291, 1317520799]
[1317513291, 1317535499]
[1317513291, 1317583499]
[1317513291, 1317587399]
[1317513291, 1317583499]
[1317513291, 1317549899]
[1317513291, 1317577499]
[1317513291, 1317586799]
[1317513291, 1317570299]
[1317513291, 1317599694]
[1317517500, 1317582299]
[1317525000, 1317585899]
[1317527100, 1317594299]
[1317529500, 1317564299]
[1317565500, 1317599694]
[1317568200, 1317599694]
[1317569100, 1317599694]
[1317570600, 1317574799]
[1317572400, 1317599694]
[1317573300, 1317599694]
[1317577200, 1317599694]
[1317577800, 1317599694]
[1317578100, 1317599694]
[1317579300, 1317599694]


In [95]:
timestamp = 1317513600
life_rank = sorted(filter(lambda x: x[1][0]>timestamp, lives.items()), key= lambda x: x[1][1], reverse= True)
life_rank

[('|id-560805', [1317565500, 1317599694]),
 ('|id-564371', [1317568200, 1317599694]),
 ('|id-562265', [1317569100, 1317599694]),
 ('|id-566431', [1317572400, 1317599694]),
 ('|id-566439', [1317573300, 1317599694]),
 ('|id-559833', [1317577200, 1317599694]),
 ('|id-566541', [1317577800, 1317599694]),
 ('|id-562374', [1317578100, 1317599694]),
 ('|id-566587', [1317579300, 1317599694]),
 ('|id-566478', [1317579600, 1317599694]),
 ('|id-566573', [1317580200, 1317599694]),
 ('|id-566602', [1317582300, 1317599694]),
 ('|id-562637', [1317583200, 1317599694]),
 ('|id-566631', [1317583500, 1317599694]),
 ('|id-566689', [1317586800, 1317599694]),
 ('|id-566726', [1317588900, 1317599694]),
 ('|id-566825', [1317593700, 1317599694]),
 ('|id-566838', [1317596100, 1317599694]),
 ('|id-566022', [1317527100, 1317594299]),
 ('|id-566013', [1317525000, 1317585899]),
 ('|id-565930', [1317517500, 1317582299]),
 ('|id-565980', [1317570600, 1317574799]),
 ('|id-566092', [1317529500, 1317564299])]

In [103]:
life_rank[:int(len(life_rank)*.3)]

[('|id-560805', [1317565500, 1317599694]),
 ('|id-564371', [1317568200, 1317599694]),
 ('|id-562265', [1317569100, 1317599694]),
 ('|id-566431', [1317572400, 1317599694]),
 ('|id-566439', [1317573300, 1317599694]),
 ('|id-559833', [1317577200, 1317599694])]

In [99]:
int(len(life_rank)*.3)

6

In [112]:
list(zip(*life_rank))

[('|id-560805',
  '|id-564371',
  '|id-562265',
  '|id-566431',
  '|id-566439',
  '|id-559833',
  '|id-566541',
  '|id-562374',
  '|id-566587',
  '|id-566478',
  '|id-566573',
  '|id-566602',
  '|id-562637',
  '|id-566631',
  '|id-566689',
  '|id-566726',
  '|id-566825',
  '|id-566838',
  '|id-566022',
  '|id-566013',
  '|id-565930',
  '|id-565980',
  '|id-566092'),
 ([1317565500, 1317599694],
  [1317568200, 1317599694],
  [1317569100, 1317599694],
  [1317572400, 1317599694],
  [1317573300, 1317599694],
  [1317577200, 1317599694],
  [1317577800, 1317599694],
  [1317578100, 1317599694],
  [1317579300, 1317599694],
  [1317579600, 1317599694],
  [1317580200, 1317599694],
  [1317582300, 1317599694],
  [1317583200, 1317599694],
  [1317583500, 1317599694],
  [1317586800, 1317599694],
  [1317588900, 1317599694],
  [1317593700, 1317599694],
  [1317596100, 1317599694],
  [1317527100, 1317594299],
  [1317525000, 1317585899],
  [1317517500, 1317582299],
  [1317570600, 1317574799],
  [1317529500, 